In [ ]:
## Activate virtual env
# $ !python3 -m virtualenv .venv
# $ !source .venv/bin/activate

## Install OpenAI package
# $ !pip install openai

## Export Gilas.io API key
# $ os.environ["GILAS_API_KEY"]='...'

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()  

client = OpenAI(
    api_key=os.environ.get("GILAS_API_KEY"),
    base_url="https://api.gilas.io/v1/"
)

In [ ]:
def get_embedding(text, dimentions=1531, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], dimensions=dimentions, model=model).data[0].embedding

In [ ]:
from scipy import spatial  # for calculating vector similarities for search
import pandas as pd

# search function
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 10
) -> tuple[list[str], list[float]]:
    
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding = get_embedding(query)
    strings_and_relatednesses = [
        (row["Text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n], relatednesses[:top_n]

In [ ]:
in_1 = "Fish are the most diverse group of vertebrates, with over 34,000 known species."
in_2 = "Some species of fish can communicate with each other using sounds, such as grunts, clicks, and chirps."
in_3 = "The largest fish is the whale shark, which can reach lengths of up to 40 feet (12 meters)."
in_4 = "Fish have been on Earth for more than 500 million years, making them one of the oldest animal groups."
in_5 = "Certain fish, like the Antarctic icefish, lack red blood cells and hemoglobin, allowing them to survive in extremely cold waters."

input_text_lst_news = [in_1, in_2, in_3, in_4, in_5]

In [ ]:
import numpy as np

data = []
for input_text in input_text_lst_news:
    data.append({"Text": input_text, "embedding": get_embedding(input_text)})
    
df = pd.DataFrame(data, columns=["Text", "embedding"])

In [ ]:
display(df.head(2))

In [ ]:
strings, relatednesses = strings_ranked_by_relatedness("Do you know if fishes can communicate with each other?", df, top_n=5)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)

In [ ]:
strings, relatednesses = strings_ranked_by_relatedness("آیا ماهی‌ها قادر به صحبت کردن یا برقراری ارتباط با هم هستن؟", df, top_n=5)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)

In [ ]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message.content

In [ ]:
## Translate user's input to English

delimiter = "####"
system_message = f"""
You are an intelligent language translator. \
Translate the message delimited with {delimiter} from Farsi to English langauge. \
Your response must solely contain the translated message without any additional comment." \
"""

user_message = f"""
آیا ماهی‌ها قادر به صحبت کردن یا برقراری ارتباط با هم هستن؟
"""

messages =  [  
{'role':'system',
 'content': system_message},   
{'role':'user',
 'content': f"""{delimiter}{user_message}{delimiter}"""},   
]
english_translation = get_completion_from_messages(messages)
print(english_translation)

In [ ]:
strings, relatednesses = strings_ranked_by_relatedness(english_translation, df, top_n=5)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)